In [3]:
import cv2
import numpy as np
import os
from loguru import logger
from tqdm import tqdm

def capture_calibration_images(save_dir, num_images=20):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        logger.error("Cannot open camera")
        exit()
    captured = 0
    while captured < num_images:
        ret, frame = cap.read()
        if not ret:
            logger.error("Failed to grab frame")
            break
        cv2.imshow('Frame', frame)
        k = cv2.waitKey(1)
        if k == ord('c'):  # Press 'c' to capture and save the image
            img_name = os.path.join(save_dir, f"calib_image_{captured}.png")
            cv2.imwrite(img_name, frame)
            logger.info(f"Captured {img_name}")
            captured += 1
        elif k == 27:  # Press 'ESC' to exit
            break
    cap.release()
    cv2.destroyAllWindows()

def find_chessboard_corners(images, chessboard_size, criteria):
    objp = np.zeros((chessboard_size[0]*chessboard_size[1], 3), np.float32)
    objp[:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1,2)

    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    return objpoints, imgpoints

def calibrate_camera(objpoints, imgpoints, frameSize):
    ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, frameSize, None, None)
    return cameraMatrix

def main():
    chessboard_size = (5, 7)  # Define the number of inner corners per a chessboard row and column
    frameSize = (640, 480)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    save_dir = './calibration_images'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Capture images for calibration
    capture_calibration_images(save_dir, num_images=20)

    images = [os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.endswith('.png')]
    objpoints, imgpoints = find_chessboard_corners(images, chessboard_size, criteria)
    cameraMatrix = calibrate_camera(objpoints, imgpoints, frameSize)

    fx = cameraMatrix[0, 0]
    fy = cameraMatrix[1, 1]
    print(f"Focal length in pixels: fx = {fx}, fy = {fy}")

if __name__ == '__main__':
    main()


[ WARN:0@598.140] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
2024-02-14 00:32:49.381 | INFO     | __main__:capture_calibration_images:23 - Captured ./calibration_images/calib_image_0.png
2024-02-14 00:32:50.600 | INFO     | __main__:capture_calibration_images:23 - Captured ./calibration_images/calib_image_1.png
2024-02-14 00:32:51.565 | INFO     | __main__:capture_calibration_images:23 - Captured ./calibration_images/calib_image_2.png
2024-02-14 00:32:52.688 | INFO     | __main__:capture_calibration_images:23 - Captured ./calibration_images/calib_image_3.png
2024-02-14 00:32:54.483 | INFO     | __main__:capture_calibration_images:23 - Captured ./calibration_images/calib_image_4.png
2024-02-14 00:32:55.912 | INFO     | __main__:capture_calibration_images:23 - Captured ./calibration_images/calib_image_5.png
2024-02-14 00:32:58.694 | INFO     | __main__:ca

Focal length in pixels: fx = 842.0157403005035, fy = 845.4201802650275


In [4]:
import os

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

# Ensure that you've correctly imported or defined MonoVideoOdometery elsewhere
from monovideoodometery import MonoVideoOdometery

focal = 4.0
pp = (0, 0)

# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(21, 21),
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01))

# Initialize video capture from the USB camera
cap = cv.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open video capture.")
    exit()

# Adjust the constructor based on your class definition
vo = MonoVideoOdometery(focal, pp, lk_params)
traj = np.zeros(shape=(600, 800, 3))

while True:
    # Read frame from camera
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Assuming you have a method to update the frame in your MonoVideoOdometery class
    vo.set_current_frame(frame)

    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:  # ESC key to exit
        break

    vo.process_frame()

    # Depending on how your MonoVideoOdometery class works, you might need to adjust this part
    # For example, if it's designed to work with a sequence of images loaded from disk,
    # you'll need to ensure it can handle continuous frames from the video stream.

    # Update trajectory visualization as needed
    # Note: Without ground truth poses, some parts related to true coordinates must be removed or modified

# Cleanup
cap.release()
cv.destroyAllWindows()


# img_path = 'C:\\Users\\Ali\\Desktop\\Projects\\SLAM\\videos\\data_odometry_gray\\dataset\\sequences\\00\\image_0\\'
# pose_path = 'C:\\Users\\Ali\\Desktop\\Projects\\SLAM\\videos\\data_odometry_poses\\dataset\\poses\\00.txt'

# focal = 718.8560
# pp = (607.1928, 185.2157)
# R_total = np.zeros((3, 3))
# t_total = np.empty(shape=(3, 1))

# # Parameters for lucas kanade optical flow
# lk_params = dict( winSize  = (21,21),
#                   criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01))


# # Create some random colors
# color = np.random.randint(0,255,(5000,3))

# vo = MonoVideoOdometery(img_path, pose_path, focal, pp, lk_params)
# traj = np.zeros(shape=(600, 800, 3))

# # mask = np.zeros_like(vo.current_frame)
# # flag = False
# while(vo.hasNextFrame()):

#     frame = vo.current_frame

#     # for i, (new,old) in enumerate(zip(vo.good_new, vo.good_old)):
#     #     a,b = new.ravel()
#     #     c,d = old.ravel()

#     #     if np.linalg.norm(new - old) < 10:
#     #         if flag:
#     #             mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
#     #             frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)


#     # cv.add(frame, mask)
#     cv.imshow('frame', frame)
#     k = cv.waitKey(1)
#     if k == 27:
#         break

#     if k == 121:
#         flag = not flag
#         toggle_out = lambda flag: "On" if flag else "Off"
#         print("Flow lines turned ", toggle_out(flag))
#         mask = np.zeros_like(vo.old_frame)
#         mask = np.zeros_like(vo.current_frame)

#     vo.process_frame()

#     print(vo.get_mono_coordinates())

#     mono_coord = vo.get_mono_coordinates()
#     true_coord = vo.get_true_coordinates()

#     print("MSE Error: ", np.linalg.norm(mono_coord - true_coord))
#     print("x: {}, y: {}, z: {}".format(*[str(pt) for pt in mono_coord]))
#     print("true_x: {}, true_y: {}, true_z: {}".format(*[str(pt) for pt in true_coord]))

#     draw_x, draw_y, draw_z = [int(round(x)) for x in mono_coord]
#     true_x, true_y, true_z = [int(round(x)) for x in true_coord]

#     traj = cv.circle(traj, (true_x + 400, true_z + 100), 1, list((0, 0, 255)), 4)
#     traj = cv.circle(traj, (draw_x + 400, draw_z + 100), 1, list((0, 255, 0)), 4)

#     cv.putText(traj, 'Actual Position:', (140, 90), cv.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255), 1)
#     cv.putText(traj, 'Red', (270, 90), cv.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255), 1)
#     cv.putText(traj, 'Estimated Odometry Position:', (30, 120), cv.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255), 1)
#     cv.putText(traj, 'Green', (270, 120), cv.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 1)

#     cv.imshow('trajectory', traj)
# cv.imwrite("./images/trajectory.png", traj)

# cv.destroyAllWindows()


ModuleNotFoundError: No module named 'monovideoodometery'

In [6]:
import numpy as np
import cv2
import glob

# Termination criteria for the corner sub-pixel optimization process
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# Adjust the following variables based on your chessboard dimensions
# Chessboard dimensions (number of inner corners per chessboard row and column)
chessboard_size = (5, 7)
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0],
                       0:chessboard_size[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

# Change the path to where your calibration images are stored
images = glob.glob('Monocular-Video-Odometery/calibration_images/*.png')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(
            gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, chessboard_size, corners2, ret)
        
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Calibrate the camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix (focal length and principal point):")
print(mtx)

Camera matrix (focal length and principal point):
[[833.23839296   0.         306.58408037]
 [  0.         836.22628495 248.90965481]
 [  0.           0.           1.        ]]


In [8]:
mtx.shape

(3, 3)

In [9]:
mtx

array([[833.23839296,   0.        , 306.58408037],
       [  0.        , 836.22628495, 248.90965481],
       [  0.        ,   0.        ,   1.        ]])

In [13]:
mtx_ = np.array([[833.23839296,   0.        , 306.58408037],
       [  0.        , 836.22628495, 248.90965481],
       [  0.        ,   0.        ,   1.        ]])

In [16]:
(mtx_[0,0]+mtx_[1,1])/2

834.732338955

In [17]:
(mtx[0, 2], mtx[1, 2])

(306.5840803736371, 248.90965481162658)